# Exercise on materials in Union
In this exercise we will focus on creation of materials with the Union components. A material describes both scattering and absorption physics, but has no geometrical shape. They will be applied to geometries in order to attribute the material properties to that geometry, but since materials can become complex, it is easier from an interface stand point to define the desired materials first and then define the system geometry.

A material will always have an absorption cross section defined using the penetration depth with units [1/m]. In addition, a material can have any number of scattering processes such as incoherent scattering, single crystal scattering and the like. First each of the desired scattering processes are defined, and next they are assembled into a material.

We start the quiz with a few questions on processes and materials.

In [ ]:
import load_quiz
quiz = load_quiz.exercise_1()

### Question 1
What does a process component do?

- A: Describes a number of scattering mechanisms
- B: Describes a physical process
- C: Describes absorption

In [ ]:
quiz.question_1("B")

Which of the following statements about a Union material is true? (created by Union_make_material)

- A: A material connects any single process to a absorption description
- B: Materials are not user defined
- C: A material collects a number of processes and absorption description
- D: The position and rotation of material components are important

In [ ]:
quiz.question_2("B")

## Create aluminium powder
Now its time to begin using process and material components.

In [ ]:
import mcstasscript as ms

In [ ]:
instrument = ms.McStas_instr("exercise_1", input_path="run_folder")
instrument.settings(output_path="data_folder/exercise_1")

 # Necessary to start with init in current version of Union in McStas 3.X
init =instrument.add_component("init", "Union_init")

### Question 3
Add an incoherent process describing aluminium.
- cross section of 4*0.0082 barns per unit cell
- unit cell volume of 66.4 Å^3

In [ ]:
# Your code here
Al_inc = instrument.add_component("Al_inc", "Incoherent_process")
Al_inc.sigma = 4*0.0082 # 4 atoms per unit cell
Al_inc.unit_cell_volume = 66.4

In [ ]:
quiz.question_3(instrument)

### Question 4
Add a powder process corresponding to aluminium.
- the McStas installation contains the datafile Al.laz

In [ ]:
# Your code here
Al_pow = instrument.add_component("Al_pow", "Powder_process")
Al_pow.reflections = '"Al.laz"'

In [ ]:
quiz.question_4(instrument)

### Look at diagram
Take a moment to look at the diagram of the instrument, it shows that the two process components are used in the material component.

In [ ]:
instrument.show_diagram()

### Question 5
Create an aluminium material called Al from the two defined processes.
- The inverse penetration depth for absorption of Al is $100 \cdot 4 \cdot 0.231$ barns $/66.4$ Å$^3$ [1/m]

The factor of 100 is from barns [m^2 E-28] $\cdot$ Å$^3$ [m^3 E-30]=[m E-2]

In [ ]:
# Your code here
Al = instrument.add_component("Al", "Union_make_material")
Al.process_string = '"Al_inc,Al_pow"'
Al.my_absorption = 100*4*0.231/66.4

In [ ]:
quiz.question_5(instrument)

## Advantages of this syntax
It might seem like a lot of code to define simple aluminium, though it is very powerful syntax as one can assemble the necessary sample through the small process pieces. Understanding this system makes it easy to make for example a twinned single crystal or add inelastic scattering onto an elastic single crystal description. A simple thing that was very difficult before the Union components was making a mixture of two powders, lets try to do so in the next questions.

Every process component has a parameter called *packing_factor* that can control how tightly it is packed by multiplying that factor onto all scattering cross sections. For the exercise we will mix gold and copper and use a parameter to control the fraction of gold.

In [ ]:
instrument.add_parameter("Au_fraction", value=0.3, comment="Fraction of Au in Au/Cu mixture")
instrument.add_declare_var("double", "Cu_fraction")
instrument.append_initialize("Cu_fraction = 1 - Au_fraction;")

### Question 6
Make an incoherent and a powder process for gold called "Au_inc" and "Au_pow" with the following properties
- 4 atoms per unit cell
- incoherent sigma: 0.43 barns
- unit cell volume: 67.86 AA^3
- reflections "Au.laz"
- packing factor Au_fraction

In [ ]:
Au_inc = instrument.add_component("Au_inc", "Incoherent_process")
Au_inc.sigma = 4*0.43 # 4 atoms per unit cell
Au_inc.unit_cell_volume = 67.86
Au_inc.packing_factor = "Au_fraction"

Au_pow = instrument.add_component("Au_pow", "Powder_process")
Au_pow.reflections = '"Au.laz"'
Au_pow.packing_factor = "Au_fraction"

In [ ]:
quiz.question_6(instrument)

### Question 7
Make an incoherent and a powder process for copper called "Cu_inc" and "Cu_pow" respectively with the following properties
- 4 atoms per unit cell
- incoherent sigma 0.55 barns
- unit cell volume 47.24 AA^3
- reflections "Cu.laz"
- packing factor Cu_fraction

In [ ]:
Cu_inc = instrument.add_component("Cu_inc", "Incoherent_process")
Cu_inc.sigma = 4*0.55 # 4 atoms per unit cell
Cu_inc.unit_cell_volume = 47.24
Cu_inc.packing_factor = "Cu_fraction"

Cu_pow = instrument.add_component("Cu_pow", "Powder_process")
Cu_pow.reflections = '"Cu.laz"'
Cu_pow.packing_factor = "Cu_fraction"

In [ ]:
quiz.question_7(instrument)

### Question 8
The mix has the sum of the absorption of each part weighted with the fraction. The absorption cross sections are quite different:
- Gold: 98.65 barns
- Copper: 3.78 barns 

The material only lacks the appropriate process string to complete it.

In [ ]:
mix = instrument.add_component("gold_and_copper_mix", "Union_make_material")
mix.process_string='"Au_inc,Au_pow,Cu_inc,Cu_pow"'
mix.my_absorption = "Au_fraction*100*4.0*98.65/67.86 + Cu_fraction*100*4.0*3.78/47.24"

In [ ]:
quiz.question_8(instrument)

## Using the material in an instrument
This exercise is about the materials, so a simple powder diffractometer is supplied which uses the "gold_and_copper_mix" material for its sample.

In [ ]:
source = instrument.add_component("source", "Source_simple")
source.set_parameters(radius=0.02, dist=6, focus_xw=0.02, focus_yh=0.02, flux=1E8)
source.lambda0 = instrument.add_parameter("double", "wavelength", value=1.2,
                                          comment="[AA]  Mean wavelength of neutrons")
source.dlambda = instrument.add_parameter("double", "d_wavelength", value=0.01,
                                          comment="[AA]  Wavelength spread of neutrons")

sample = instrument.add_component("sample", "Union_cylinder")
sample.set_parameters(yheight=0.02, radius=0.008,
                      material_string='"gold_and_copper_mix"', priority=1)
sample.set_AT(5.5, RELATIVE=source)

instrument.add_component("master", "Union_master")
instrument.add_component("stop", "Union_stop")

banana = instrument.add_component("banana", "Monitor_nD", RELATIVE=sample)
banana.xwidth = 2.0
banana.yheight = 0.3
banana.restore_neutron = 1
banana.filename = '"banana.dat"'
banana.options = '"theta limits=[5 175] bins=341, banana"'

psd = instrument.add_component("psd_lin", "PSDlin_monitor")
psd.set_parameters(nbins=100, xwidth=0.02, yheight=0.02, filename='"PSD.dat"')
psd.set_AT(6, RELATIVE=source)

trans = instrument.add_component("transmission", "L_monitor")
trans.set_parameters(nL=100, xwidth=0.02, yheight=0.02,
                     Lmin="wavelength-d_wavelength", Lmax="wavelength+d_wavelength",
                     filename='"transmission.dat"')
trans.set_AT(0, RELATIVE=psd)

### Check diagram
With the diagram we can see how this Union system is connected.

In [ ]:
instrument.show_diagram()

## Run instrument
We can use the widget interface to run the instrument and see how the resulting powder pattern changes with the composition of the mixture. You can also look for Bragg edges in the wavelength resolved transmission data.

In [ ]:
import mcstasscript.jb_interface as ms_widget
%matplotlib widget

In [ ]:
ms_widget.show(instrument)